In [86]:
import requests
from bs4 import BeautifulSoup

from bs4.element import Comment

import pandas as pd
import re
from tqdm import trange
from pathlib import Path

In [144]:
def extract_contents(topic):
    df = pd.read_csv(f'data/titles/{topic}.csv')
    urls = []
    contents = []
    for idx, row in df.iterrows():
        title = row['title']   
        page = wikipedia.page(title)
        urls.append(page.url)
        contents.append(page.content)
    
    df['url'] = urls
    df['content'] = contents
    df.to_csv(f'data/contents/{topic}.csv', index=False)
    
# extract_contents('personas')
# extract_contents('science')
# extract_contents('technology')
# extract_contents('aircrafts')
extract_contents('history')

In [141]:
# make unique
def process_titles(topic):
    in_df = pd.read_csv(f'data/titles/_{topic}.csv', header=None)
    titles = in_df[0].to_list()
    titles = list(set(titles))
    titles = sorted(titles)
    out_df = pd.DataFrame(titles, columns=['title'])
    out_df.to_csv(f'data/titles/{topic}.csv', index=False)
    
# process_titles('science')
# process_titles('technology')
# process_titles('aircrafts')
process_titles('history')

In [145]:
def group_contents(topics):
    df = pd.DataFrame(columns=['title', 'url', 'content', 'topic'])
    for t in topics:
        tdf = pd.read_csv(f'data/contents/{t}.csv')
        tdf['topic']=t
        print(tdf.columns)
        df = df.append(tdf, ignore_index=True)
    df.to_csv(f'data/contents/total.csv', index=False)
    return df

topics = ['science','technology','aircrafts','history','personas','food']
group_contents(topics)
# group_contents(['history'])


Index(['title', 'url', 'content', 'topic'], dtype='object')
Index(['title', 'url', 'content', 'topic'], dtype='object')
Index(['title', 'url', 'content', 'topic'], dtype='object')
Index(['title', 'url', 'content', 'topic'], dtype='object')
Index(['title', 'url', 'content', 'topic'], dtype='object')
Index(['title', 'url', 'content', 'topic'], dtype='object')


,title,url,content,topic
0,Sotsionika,https://tt.wikipedia.org/wiki/Sotsionika,Sotsionika - ğämäli psixologiädäge yünäleş. Ul...,science
1,Агрономия,https://tt.wikipedia.org/wiki/%D0%90%D0%B3%D1%...,"Агрономия (грек. agros - кыр, nomos - канун) —...",science
2,Акционерлар җәмгыятьләре‎,https://tt.wikipedia.org/wiki/%D0%AF%D0%B4%D0%...,"Ядернә — (рус. Ядрин, тау мари телендә Йӓдӹрнӓ...",science
3,Алюминий композит панелләре,https://tt.wikipedia.org/wiki/%D0%90%D0%BB%D1%...,Алюминий композит панелләре – каплый торган ко...,science
4,Архитектура,https://tt.wikipedia.org/wiki/%D0%90%D1%80%D1%...,Архитекту́ра яки мигъмарият (лат. architectura...,science
...,...,...,...,...
434,Шишара,https://tt.wikipedia.org/wiki/%D0%A8%D0%B8%D1%...,Шишара - төрле-төрле фигуралар формасындагы ри...,food
435,Шәнгә,https://tt.wikipedia.org/wiki/%D0%A8%D3%99%D0%...,"Шәнгә — бик борынгы ашамлык, аны бигрәк тә Се...",food
436,Юача,https://tt.wikipedia.org/wiki/%D0%AE%D0%B0%D1%...,Юача - милли ризык.\nЮачаны әче камырдан ясыйл...,food
437,Җылгыр,https://tt.wikipedia.org/wiki/%D2%96%D1%8B%D0%...,Җылгыр — киндер орлыгының оны белән әзерләнә т...,food


In [ ]:
from bs4 import BeautifulSoup
import requests
import urllib

page_url = 'https://tt.wikipedia.org/wiki/%D0%9C%D0%B0%D1%85%D1%81%D1%83%D1%81:%D0%A2%D3%A9%D1%80%D0%BA%D0%B5%D0%BC%D0%BD%D3%99%D1%80'



def process_cat(page_url):
    cont = requests.get(page_url).content
    soup = BeautifulSoup(cont)
    
    articles_block = soup.find('div', dir='ltr', class_='mw-content-ltr')
    t = articles_block.find_all('a', class_=None)
    
    print(t)

    
def extract_anchors(url):
    print(url)
    cont = requests.get(url).content
    soup = BeautifulSoup(cont)
    
    anchors = {anchor.text:urllib.parse.urljoin(page_url, anchor.get('href')) for anchor in soup.find_all('a')}
    try:
        anchors.update(extract_anchors(anchors['киләсе 50']))
    except KeyError:
        pass

    return anchors

    # for a in anchors[:20]:
#     print(a)
# print(anchors[:-5])    

# process_cat('https://tt.wikipedia.org/wiki/%D0%A2%D3%A9%D1%80%D0%BA%D0%B5%D0%BC:%22%D0%92%D0%B0%D1%82%D0%B0%D0%BD_%D0%B0%D0%BB%D0%B4%D1%8B%D0%BD%D0%B4%D0%B0_%D0%BA%D0%B0%D0%B7%D0%B0%D0%BD%D1%8B%D1%88%D0%BB%D0%B0%D1%80%D1%8B_%D3%A9%D1%87%D0%B5%D0%BD%22_%D0%BE%D1%80%D0%B4%D0%B5%D0%BD%D1%8B_%D0%BA%D0%B0%D0%B2%D0%B0%D0%BB%D0%B5%D1%80%D0%BB%D0%B0%D1%80%D1%8B')
extract_anchors(page_url)

In [140]:

p = wikipedia.page("аяк")
p.categories

['Төркем:Сөяк системасы']

In [ ]:
import wikipedia
wikipedia.set_lang('tt')

data_item = 
p = wikipedia.page("бавырсак")
p = wikipedia.page("Портал:Порталлар")
print(p.url)
print(p.title)
print(p.summary)

print(wikipedia.WikipediaPage.__dict__)
content = p.content # Content of page.
content

In [171]:
   

# Here I will try transliteration
rules = {
'ts':'ц',
'yü':'ю',
'yu':'ю',
'eu':'ев',
'a':'а',
'b':'б',
'ç':'ч',
'c':'җ',
'd':'д',
'e':'е',
'ä':'ә',
'f':'ф',
'g':'г',
'ğ':'г',
'h':'һ',
'i':'и',
'y':'й',
'k':'к',
'l':'л',
'm':'м',
'n':'н',
'ñ':'ң',
'o':'о',
'ö':'ө',
'p':'п',
'q':'к',
'r':'р',
's':'с',
'ş':'ш',
't':'т',
'u':'у',
'v':'в',
'w':'в',
'x':'х',
'ü':'ү',
'z':'з',
'j':'ж',
'ı':'ы',
'\'':'(ъ, ь, э)',
'í':'ый',
    }

In [172]:
import re
page = wikipedia.page('Sotsionika')
txt = page.content
txt =txt.lower()
for letter in rules.keys():
    txt = re.sub(f'{letter}',f'{rules[letter]}', txt)

print(page.content)
print(txt)

Sotsionika - ğämäli psixologiädäge yünäleş. Ul Litva sotsiologı Auşra Augustinaviçüte (Augusta pseudonimı astında yazılğan) gipotezasına nigezlängän. Bu gipoteza buyınça keşelär arasındağı psixologik täñgällek, alarnıñ nindi psixologik törgä qarawına bäyle. Augusta tör-ara mönäsäbätlär modelen  Şweytsariä psixologı Karl Yung taswirlawınnan çığıp tözegän. 
Sotsionikanın köçle yağı - anıñ belän köndäş bulırday başqa yünäleşneñ bulmawında. Psixologiädä älege başqa berkemneñ dä psixologik täñgällek teoriäsen täqdim itkäne yuq. Şul uq waqıtta, sotsionika artıq küp küreneşlärne Karl Yung psixologiäse qısalarında anlatırğa tırışa, şul isäptän, anda fänni elementlardan tış, fänni bulmağannarı da küp.


== Päräwäzdä ==
[1]
соционика - гәмәли психологиәдәге юнәлеш. ул литва социологы аушра аугустинавичүте (аугуста псевдонимы астында йазылган) гипотезасына нигезләнгән. бу гипотеза буйынча кешеләр арасындагы психологик тәңгәллек, аларның нинди психологик төргә каравына бәйле. аугуста төр-ара мөнәс

In [95]:
BASE_URL = 'https://tt.wikipedia.org'

def is_category_tag(tag):
    match = re.search(r'\(\d+\sәгъза\)', tag.text)
    return match is not None

def extract_count(txt):
    match = re.search(r'\(\d+\sәгъза\)', txt)
    cnt = match.group(0)
    cnt = re.sub(r'[\(\)]', '', cnt).split()[0]
    return int(cnt)

def process_title(title):
    title = re.sub(r'Төркем:', '',title)
    return title

def process_single(page_url):
    cont = requests.get(page_url).content
    soup = BeautifulSoup(cont)
    
    items = soup.find_all('li')
    items = [it for it in items if is_category_tag(it)]
    
    df = pd.DataFrame(columns = ['link', 'text'])
    df['link'] = [item.find('a') for item in items]
    
    df['title'] = [link.get('title') for link in df['link']]
    df['url'] = [BASE_URL+link.get('href') for link in df['link']]
    
    df['text'] = [item.text for item in items]
    df['count'] = [extract_count(txt) for txt in df['text']]
    
    # filtering
    df = df[df['title'].str.contains('Төркем:')]
    df = df[df['url'].str.contains('/wiki/')]
    
    # only columns we need
    df = df[['title','url','count']]
    
    # postprocessing
    df['title'] = [process_title(title) for title in df['title']]
    
    return df

def next_page_url(page_url):
    cont = requests.get(page_url).content
    soup = BeautifulSoup(cont)
    
    tags = soup.find_all('a', text='киләсе 5000')
    if len(tags)==0:
        return None
    
    return BASE_URL+tags[0].get('href')
    

def extract_categories(start_url, out_path:Path(), limit=10):
    df = pd.DataFrame(columns = ['title','url','count'])
    df.to_csv(out_path, mode='a', index=False)
    current_url = start_url
    for i in trange(limit):
        if current_url is None:
            return i

        df = process_single(current_url)
        df.to_csv(out_path, mode='a', header=False, index=False)
        
        current_url= next_page_url(current_url)
    
    return limit

limit = '10000'
start_url = f'https://tt.wikipedia.org/w/index.php?title=%D0%9C%D0%B0%D1%85%D1%81%D1%83%D1%81:%D0%A2%D3%A9%D1%80%D0%BA%D0%B5%D0%BC%D0%BD%D3%99%D1%80&offset=&limit={limit}'
# start_url = f'https://tt.wikipedia.org/w/index.php?title=%D0%9C%D0%B0%D1%85%D1%81%D1%83%D1%81:%D0%A2%D3%A9%D1%80%D0%BA%D0%B5%D0%BC%D0%BD%D3%99%D1%80&offset=1992_%D0%B5%D0%BB%D0%B4%D0%B0_%D0%B1%D0%B0%D1%80%D0%BB%D1%8B%D0%BA%D0%BA%D0%B0_%D0%BA%D0%B8%D0%BB%D0%B3%D3%99%D0%BD_%D0%B1%D0%B0%D1%81%D0%BC%D0%B0%D0%BB%D0%B0%D1%80&limit=5000'
# df = process_single(start_url)

# df.tail(20)
# next_page_url(start_url)
cat_path = Path('cats.csv')
count = extract_categories(start_url,cat_path, limit=10)
count

 70%|███████   | 7/10 [00:55<00:23,  7.95s/it]


7

In [99]:
df = pd.read_csv('cats.csv')

df = df[df['count']>0]
df = df.sort_values('count', ascending=False)
t = df['url'].head(10)
t = list(t)
print('\n'.join(t))

https://tt.wikipedia.org/wiki/%D0%A2%D3%A9%D1%80%D0%BA%D0%B5%D0%BC:Commons_category_link_is_on_Wikidata
https://tt.wikipedia.org/wiki/%D0%A2%D3%A9%D1%80%D0%BA%D0%B5%D0%BC:%D3%98%D0%BB%D0%B8%D1%84%D0%B1%D0%B0_%D0%B1%D1%83%D0%B5%D0%BD%D1%87%D0%B0_%D1%82%D0%BE%D1%80%D0%B0%D0%BA_%D0%BF%D1%83%D0%BD%D0%BA%D1%82%D0%BB%D0%B0%D1%80
https://tt.wikipedia.org/wiki/%D0%A2%D3%A9%D1%80%D0%BA%D0%B5%D0%BC:%D0%9A%D0%B8%D1%80%D0%BE%D0%B2_%D3%A9%D0%BB%D0%BA%D3%99%D1%81%D0%B5_%D1%82%D0%BE%D1%80%D0%B0%D0%BA_%D0%BF%D1%83%D0%BD%D0%BA%D1%82%D0%BB%D0%B0%D1%80%D1%8B
https://tt.wikipedia.org/wiki/%D0%A2%D3%A9%D1%80%D0%BA%D0%B5%D0%BC:%D0%91%D0%B0%D1%88%D0%BA%D0%BE%D1%80%D1%82%D1%81%D1%82%D0%B0%D0%BD_%D0%B0%D0%B2%D1%8B%D0%BB%D0%BB%D0%B0%D1%80%D1%8B
https://tt.wikipedia.org/wiki/%D0%A2%D3%A9%D1%80%D0%BA%D0%B5%D0%BC:%D0%A5%D1%80%D0%BE%D0%BD%D0%BE%D0%BB%D0%BE%D0%B3%D0%B8%D0%BA_%D0%B8%D1%81%D0%B5%D0%BC%D0%BB%D0%B5%D0%BA%D0%BB%D3%99%D1%80
https://tt.wikipedia.org/wiki/%D0%A2%D3%A9%D1%80%D0%BA%D0%B5%D0%BC:%D0%92%D0%B8%D0

In [101]:
df.head(50)

,title,url,count
10129,Commons category link is on Wikidata,https://tt.wikipedia.org/wiki/%D0%A2%D3%A9%D1%...,9963
24368,Әлифба буенча торак пунктлар,https://tt.wikipedia.org/wiki/%D0%A2%D3%A9%D1%...,7514
15942,Киров өлкәсе торак пунктлары,https://tt.wikipedia.org/wiki/%D0%A2%D3%A9%D1%...,4590
12050,Башкортстан авыллары,https://tt.wikipedia.org/wiki/%D0%A2%D3%A9%D1%...,4185
22337,Хронологик исемлекләр,https://tt.wikipedia.org/wiki/%D0%A2%D3%A9%D1%...,3926
12994,Википедия:Викиҗыентыкта үз төркеме булмаган то...,https://tt.wikipedia.org/wiki/%D0%A2%D3%A9%D1%...,3842
16035,Коми Республикасы елгалары,https://tt.wikipedia.org/wiki/%D0%A2%D3%A9%D1%...,3813
10239,No local image but image on Wikidata,https://tt.wikipedia.org/wiki/%D0%A2%D3%A9%D1%...,3459
13957,Еллар,https://tt.wikipedia.org/wiki/%D0%A2%D3%A9%D1%...,3126
23720,Ямал-Ненец АО елгалары,https://tt.wikipedia.org/wiki/%D0%A2%D3%A9%D1%...,2826
